In [1]:
from asyncio import get_event_loop
from twisted.internet import asyncioreactor

asyncioreactor.install(get_event_loop())

In [2]:
from twisted.internet import reactor
from twisted.internet.task import CooperativeTask, Cooperator, TaskDone, TaskFailed
from twisted.internet.defer import ensureDeferred, Deferred
from twisted.python.failure import Failure

In [3]:
class AsyncCooperativeTask(CooperativeTask):
    """
    An experiment in making a CooperativeTask that works
    with async iterators.
    """
    
    async def _oneAsyncWorkUnit(self):
        try:
            self.pause()
            result = await self._iterator.__anext__()
        except StopAsyncIteration:
            self._completeWith(TaskDone(), self._iterator)
        except:
            self._completeWith(TaskFailed(), Failure())
        else:
            self.resume()
    
    def _oneWorkUnit(self):
        result = ensureDeferred(self._oneAsyncWorkUnit())


class AsyncCooperator(Cooperator):
    """
    An experiment in making a Cooperator that can work with
    async iterators.
    """
    
    def coiterate(self, asyncIterator, doneDeferred=None):
        if doneDeferred is None:
            doneDeferred = Deferred()
        AsyncCooperativeTask(asyncIterator, self).whenDone().chainDeferred(doneDeferred)
        return doneDeferred
    
    def cooperate(self, asyncIterator):
        return AsyncCooperativeTask(asyncIterator, self)

In [4]:
def sleep(t):
    d = Deferred()
    reactor.callLater(t, d.callback, None)
    return d


async def testGenerator():
    print('first we nap for a second')
    await sleep(1)
    print('then we yield')
    
    yield
    
    print('then we sleep some more...')
    
    await sleep(1)
    
    print('then we yield some more!')
    
    yield
    
    print('one more nap...')
    
    await sleep(1)
    
    print('we are done.')
    
    return
    

In [5]:
cooperator = AsyncCooperator()

d = cooperator.coiterate(testGenerator())

d.addCallback(lambda task: print('success!'))

<Deferred at 0x7fa3db0106a0 waiting on Deferred at 0x7fa3db010860>

first we nap for a second
then we yield
then we sleep some more...
then we yield some more!
one more nap...
we are done.
success!
